In [175]:
#!pip install kafka-python
#!pip install faker
# currently using kafka-python-2.0.2, Faker-28.0.0
from kafka import KafkaProducer
from kafka import KafkaConsumer
import sys
import time
import random
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import datetime as tm
from faker import Faker

In [176]:
#Kafka source details - this is for EDF Kafka Wire Protocol - so the broker is where Data Access Gateway is running and its port
server = "172.31.37.92:9092"
topic = "freshtopic"
print("BROKER: " + server)
print("TOPIC: " + topic)

BROKER: 172.31.37.92:9092
TOPIC: freshtopic


In [177]:
# this API call will change based on the actual Kafka broker 
#PRODUCER
print("\n**Starting Producer**")
producer=KafkaProducer(bootstrap_servers=[server],
                        security_protocol='SASL_PLAINTEXT',
                        sasl_mechanism='PLAIN',
                        sasl_plain_username='mapr',
                        sasl_plain_password='mapr')

<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <connecting> [IPv4 ('172.31.37.92', 9092)]>: connecting to 172.31.37.92:9092 [('172.31.37.92', 9092) IPv4]
Probing node bootstrap-0 broker version
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Sending username and password in the clear
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Authenticated as mapr via PLAIN
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Connection complete.



**Starting Producer**


Broker version identified as 2.4.0
Set configuration api_version=(2, 4, 0) to skip auto check_version requests on startup
<BrokerConnection node_id=0 host=ip-172-31-40-191.ap-southeast-2.compute.internal:9092 <connected> [IPv4 ('172.31.40.191', 9092)]>: Closing connection. 


In [179]:
numMsgProduced = 0
fake = Faker()
for h in range(100):
    record = {
        "id": h+random.randint(1,100000),
        "name": fake.name(),
        "address": fake.address(),
        "amount": random.randint(1,1000000)
    }
    print(record)
    producer.send(topic, json.dumps(record).encode('utf-8'))
    numMsgProduced += 1
producer.flush()
print("Messages produced: " + str(numMsgProduced))
time.sleep(2)

{'id': 22082, 'name': 'Christine Rogers', 'address': 'Unit 2061 Box 5294\nDPO AP 38515', 'amount': 289376}
{'id': 56413, 'name': 'Brittany Green', 'address': '26590 Clark Shoals\nChristopherfurt, MS 52759', 'amount': 519002}
{'id': 22543, 'name': 'Ashley Johnson', 'address': 'USS Bullock\nFPO AA 72680', 'amount': 556919}
{'id': 47289, 'name': 'Bobby Hebert', 'address': '237 Joseph Stravenue\nFloresstad, FM 24295', 'amount': 408452}
{'id': 3541, 'name': 'Lauren Bell', 'address': '32655 Daniel Crossroad\nNew Brianshire, AL 67960', 'amount': 214789}
{'id': 53413, 'name': 'Joshua Horton', 'address': '309 Darlene Extension\nNorth Davidmouth, DE 18896', 'amount': 680200}
{'id': 46572, 'name': 'Anthony Smith', 'address': '9985 Cook Estate Apt. 703\nNorth Anaville, PR 33163', 'amount': 84565}
{'id': 24917, 'name': 'April Freeman', 'address': '01037 Ashley Cove Apt. 831\nNew Jessicaborough, ND 01436', 'amount': 130820}
{'id': 95676, 'name': 'Gabrielle Rasmussen', 'address': '2404 Barry Point\nN

In [109]:
# this will keep track of records read and will ignore them in re-reading data
# Do NOT run this cell unless you want to go back to beginning for ingestion
consumer_offset = -1

In [110]:
#list of records received
fullrecords=[]
datarecords=[]

In [111]:
# CONSUMER
print("\n**Starting Consumer**")
consumer = KafkaConsumer(bootstrap_servers=[server],
                         auto_offset_reset='earliest',
                        security_protocol='SASL_PLAINTEXT',
                        sasl_mechanism='PLAIN',
                        sasl_plain_username='mapr',
                        sasl_plain_password='mapr')

consumer.subscribe(topic)
numMsgConsumed = 0

for _ in range(10):
    records = consumer.poll(timeout_ms=500)
    for topic_data, consumer_records in records.items():
        for consumer_record in consumer_records:
            if(consumer_record.offset > consumer_offset):
                print("Offset: " + str(consumer_record.offset) + "  Received message: " + consumer_record.value.decode('utf-8'))
                fullrecords.append(consumer_record)
                datarecords.append(json.loads(consumer_record.value.decode('utf-8')))
                consumer_offset = consumer_record.offset
                numMsgConsumed += 1
            else:
                print("Offset: " + str(consumer_record.offset) + " Already consumed")
print("New Messages consumed: " + str(numMsgConsumed))

<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <connecting> [IPv4 ('172.31.37.92', 9092)]>: connecting to 172.31.37.92:9092 [('172.31.37.92', 9092) IPv4]
Probing node bootstrap-0 broker version
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Sending username and password in the clear
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Authenticated as mapr via PLAIN
<BrokerConnection node_id=bootstrap-0 host=172.31.37.92:9092 <authenticating> [IPv4 ('172.31.37.92', 9092)]>: Connection complete.



**Starting Consumer**


Broker version identified as 2.4.0
Set configuration api_version=(2, 4, 0) to skip auto check_version requests on startup
group_id is None: disabling auto-commit.
Updating subscribed topics to: ['freshtopic']
Updated partition assignment: [TopicPartition(topic='freshtopic', partition=0)]
<BrokerConnection node_id=0 host=ip-172-31-40-191.ap-southeast-2.compute.internal:9092 <connecting> [IPv4 ('172.31.40.191', 9092)]>: connecting to ip-172-31-40-191.ap-southeast-2.compute.internal:9092 [('172.31.40.191', 9092) IPv4]
<BrokerConnection node_id=0 host=ip-172-31-40-191.ap-southeast-2.compute.internal:9092 <authenticating> [IPv4 ('172.31.40.191', 9092)]>: Sending username and password in the clear
<BrokerConnection node_id=0 host=ip-172-31-40-191.ap-southeast-2.compute.internal:9092 <authenticating> [IPv4 ('172.31.40.191', 9092)]>: Authenticated as mapr via PLAIN
<BrokerConnection node_id=0 host=ip-172-31-40-191.ap-southeast-2.compute.internal:9092 <authenticating> [IPv4 ('172.31.40.191', 90

Offset: 0  Received message: {"id": 43946, "name": "Justin Guerra", "address": "954 Banks Forges\nHawkinsberg, MH 76341", "amount": 181384}
Offset: 1  Received message: {"id": 85853, "name": "Katelyn Hernandez", "address": "5795 Mackenzie Fields Apt. 911\nLaurahaven, OH 08329", "amount": 37373}
Offset: 2  Received message: {"id": 48532, "name": "Diane Harris PhD", "address": "2535 Schneider Rue Suite 835\nKruegershire, VI 71315", "amount": 629494}
Offset: 3  Received message: {"id": 37061, "name": "Cassidy Skinner", "address": "536 Joshua Forge\nCarterfort, MI 55439", "amount": 187243}
Offset: 4  Received message: {"id": 15468, "name": "Antonio Perez", "address": "USNV Jacobs\nFPO AE 48706", "amount": 733108}
Offset: 5  Received message: {"id": 80154, "name": "Paula Monroe", "address": "40373 Justin Pines Apt. 251\nEricfort, IA 97181", "amount": 260877}
Offset: 6  Received message: {"id": 64766, "name": "Kevin Byrd", "address": "9389 Rodney Crest Suite 361\nPort Lisaton, AZ 45944", "am

In [112]:
# check offset is tracked correctly
consumer_offset

399

In [122]:
#build pandas data frame from kafka consumer ingested records
optable = pa.Table.from_pandas(pd.DataFrame(datarecords))

In [139]:
uniquestr=tm.datetime.now().strftime("%Y-%m-%d-%H-%M")

In [140]:
pq.write_table(optable,"/mnt/shared/ingest1/source1/kafkaingest1"+uniquestr+".parquet")